# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
!pip install pandas matplotlib numpy citipy geopy requests hvplot
!pip install geoviews

                                              0.0/155.4 kB ? eta -:--:--
     -----------------------                 92.2/155.4 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 155.4/155.4 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,10.29,61,92,1.34,AU,1721774850
1,1,remire-montjoly,4.9167,-52.2667,28.02,78,0,1.54,GF,1721774851
2,2,angaur state,6.9060,134.1300,25.63,92,100,6.02,PW,1721774852
3,3,waitangi,-43.9535,-176.5597,9.34,99,100,4.47,NZ,1721774853
4,4,angra do heroismo,38.6500,-27.2167,24.34,83,20,0.45,PT,1721774854


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='City',
    alpha=0.5,
    frame_width=800,
    frame_height=600,
    hover_cols=['City']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,ain taya,36.7933,3.2869,22.91,73,0,1.54,DZ,1721774855
53,53,navoiy,40.0844,65.3792,24.71,29,0,3.09,UZ,1721774910
73,73,nova vicosa,-17.8919,-39.3719,21.51,68,0,3.23,BR,1721774932
183,183,baskil,38.5679,38.8238,24.91,42,0,1.79,TR,1721775059
232,232,lata,40.1629,-8.3327,26.07,54,0,0.63,PT,1721775114
265,265,armacao de buzios,-22.7469,-41.8817,21.03,81,0,1.53,BR,1721774904
279,279,barcelos,41.5388,-8.6151,21.08,54,0,1.78,PT,1721775170
281,281,ain el bya,35.8039,-0.3018,25.27,63,0,1.50,DZ,1721775172
348,348,ararat,39.8317,44.7049,24.19,44,0,3.83,AM,1721775247
452,452,urucui,-7.2294,-44.5561,26.58,37,0,2.23,BR,1721775368


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,ain taya,DZ,36.7933,3.2869,73,
53,navoiy,UZ,40.0844,65.3792,29,
73,nova vicosa,BR,-17.8919,-39.3719,68,
183,baskil,TR,38.5679,38.8238,42,
232,lata,PT,40.1629,-8.3327,54,
265,armacao de buzios,BR,-22.7469,-41.8817,81,
279,barcelos,PT,41.5388,-8.6151,54,
281,ain el bya,DZ,35.8039,-0.3018,63,
348,ararat,AM,39.8317,44.7049,44,
452,urucui,BR,-7.2294,-44.5561,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ain taya - nearest hotel: فندق
navoiy - nearest hotel: Biryuza
nova vicosa - nearest hotel: Pousada Pontal da Barra
baskil - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
armacao de buzios - nearest hotel: Vila da Santa
barcelos - nearest hotel: Hotel Dom Nuno
ain el bya - nearest hotel: فندق أريحا
ararat - nearest hotel: No hotel found
urucui - nearest hotel: Formula Flat hotel
albert lea - nearest hotel: Country Inn & Suites, Albert Lea, MN
puerto quijarro - nearest hotel: No hotel found
playa del ingles - nearest hotel: Apartamentos Barbados
ardesen - nearest hotel: No hotel found
pamandzi - nearest hotel: Gite Le Madina
bonito - nearest hotel: Hotel Pousada Arauna
ipora - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
5,ain taya,DZ,36.7933,3.2869,73,فندق
53,navoiy,UZ,40.0844,65.3792,29,Biryuza
73,nova vicosa,BR,-17.8919,-39.3719,68,Pousada Pontal da Barra
183,baskil,TR,38.5679,38.8238,42,No hotel found
232,lata,PT,40.1629,-8.3327,54,Residencial Botânico
265,armacao de buzios,BR,-22.7469,-41.8817,81,Vila da Santa
279,barcelos,PT,41.5388,-8.6151,54,Hotel Dom Nuno
281,ain el bya,DZ,35.8039,-0.3018,63,فندق أريحا
348,ararat,AM,39.8317,44.7049,44,No hotel found
452,urucui,BR,-7.2294,-44.5561,37,Formula Flat hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    color='City',
    alpha=0.5,
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)